In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive

# Import and function definition

In [ ]:
# Download and import visualkeras and keras-tuner library
!pip install visualkeras
!pip install keras-tuner -q
!pip install pillow

In [ ]:
import visualkeras
import tensorflow as tf
import math
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from keras import Model
from PIL import Image
import keras_tuner
from sklearn.model_selection import KFold
from IPython.display import clear_output

In [ ]:
IMG_SIZE = 96
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 512


def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)


@tf.function
def get_box(lambda_value):
    cut_rat = tf.math.sqrt(1.0 - lambda_value)

    cut_w = IMG_SIZE * cut_rat  # rw
    cut_w = tf.cast(cut_w, tf.int32)

    cut_h = IMG_SIZE * cut_rat  # rh
    cut_h = tf.cast(cut_h, tf.int32)

    cut_x = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # rx
    cut_y = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # ry

    boundaryx1 = tf.clip_by_value(cut_x[0] - cut_w // 2, 0, IMG_SIZE)
    boundaryy1 = tf.clip_by_value(cut_y[0] - cut_h // 2, 0, IMG_SIZE)
    bbx2 = tf.clip_by_value(cut_x[0] + cut_w // 2, 0, IMG_SIZE)
    bby2 = tf.clip_by_value(cut_y[0] + cut_h // 2, 0, IMG_SIZE)

    target_h = bby2 - boundaryy1
    if target_h == 0:
        target_h += 1

    target_w = bbx2 - boundaryx1
    if target_w == 0:
        target_w += 1

    return boundaryx1, boundaryy1, target_h, target_w


@tf.function
def cutmix(train_ds_one, train_ds_two):
    (image1, label1), (image2, label2) = train_ds_one, train_ds_two

    alpha = [0.25]
    beta = [0.25]

    # Get a sample from the Beta distribution
    lambda_value = sample_beta_distribution(1, alpha, beta)

    # Define Lambda
    lambda_value = lambda_value[0][0]

    # Get the bounding box offsets, heights and widths
    boundaryx1, boundaryy1, target_h, target_w = get_box(lambda_value)

    # Get a patch from the second image (`image2`)
    crop2 = tf.image.crop_to_bounding_box(
        image2, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image2` patch (`crop2`) with the same offset
    image2 = tf.image.pad_to_bounding_box(
        crop2, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )
    # Get a patch from the first image (`image1`)
    crop1 = tf.image.crop_to_bounding_box(
        image1, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image1` patch (`crop1`) with the same offset
    img1 = tf.image.pad_to_bounding_box(
        crop1, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )

    # Modify the first image by subtracting the patch from `image1`
    # (before applying the `image2` patch)
    image1 = image1 - img1
    # Add the modified `image1` and `image2`  together to get the CutMix image
    image = image1 + image2

    # Adjust Lambda in accordance to the pixel ration
    lambda_value = 1 - (target_w * target_h) / (IMG_SIZE * IMG_SIZE)
    lambda_value = tf.cast(lambda_value, tf.float32)

    # Combine the labels of both images
    label = lambda_value * label1 + (1 - lambda_value) * label2
    return image, label

def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.image.convert_image_dtype(image, tf.float32) / 255.0
    return image, label

In [ ]:
def load_data(path, labels):
    # Dataset folders
    dataset_dir = path

    gen = ImageDataGenerator().flow_from_directory(directory=dataset_dir,
                                               target_size=(IMG_SIZE,IMG_SIZE),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels or None
                                               class_mode='categorical',
                                               batch_size=batchSize,
                                               shuffle=True,
                                               seed=seed)
    

    return gen

def processing_data(data, labels):

    X,Y = data.next()
    
    check_image = os.path.exists('/gdrive/My Drive/images.npy')
    check_target = os.path.exists('/gdrive/My Drive/targets.npy')
    if(check_image and check_target):
      X = np.load('images.npy')
      Y = np.load('targets.npy')
    else:
      for i in tqdm(range(data.__len__())):
        images, targets = data.next()
        X = np.concatenate((X, images), axis = 0)
        Y = np.concatenate((Y, targets), axis = 0)
      np.save('images.npy', X)
      np.save('targets.npy', Y)
    
    
    Xtest = X[0]
    Ytest = Y[0]
    Xtrain = X[0]
    Ytrain = Y[0]
    Xval = X[0]
    Yval = Y[0]
    XvalTuner = X[0]
    YvalTuner = Y[0]

    print("total size: "+str(X.shape)+" "+str(Y.shape))

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=paramSplit, random_state=seed, stratify=Y)
    Xtest, Xval, Ytest, Yval = train_test_split(Xtest,Ytest, test_size=(2/3), random_state=seed, stratify=Ytest)
    Xval, XvalTuner, Yval, YvalTuner = train_test_split(Xval,Yval, test_size=0.5, random_state=seed, stratify=Yval)

    #Normalize the data like supernet
    Xtrain = tf.keras.applications.vgg16.preprocess_input(Xtrain)
    Xval = tf.keras.applications.vgg16.preprocess_input(Xval)
    XvalTuner = tf.keras.applications.vgg16.preprocess_input(XvalTuner)
    Xtest = tf.keras.applications.vgg16.preprocess_input(Xtest)

    shapeBeforeDA = str(Xtrain.shape)+" "+str(Ytrain.shape)

    #Data augmentation cutmix
    check_image = os.path.exists('/gdrive/My Drive/imagesDA.npy')
    check_target = os.path.exists('/gdrive/My Drive/targetsDA.npy')
    if(check_image and check_target):
       Xtrain = np.load('imagesDA.npy')
       Ytrain = np.load('targetsDA.npy')
    else:

      train_ds_one = (
          tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain))
          .shuffle(1024)
          .map(preprocess_image, num_parallel_calls=AUTO)
      )
      train_ds_two = (
          tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain))
          .shuffle(1024)
          .map(preprocess_image, num_parallel_calls=AUTO)
      )

      train_ds_simple = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain))

      test_ds = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain))

      train_ds_simple = (
          train_ds_simple.map(preprocess_image, num_parallel_calls=AUTO)
          .batch(BATCH_SIZE)
          .prefetch(AUTO)
      )

      # Combine two shuffled datasets from the same training data.
      train_ds = tf.data.Dataset.zip((train_ds_one, train_ds_two))

      test_ds = (
          test_ds.map(preprocess_image, num_parallel_calls=AUTO)
          .batch(BATCH_SIZE)
          .prefetch(AUTO)
      )

      train_ds_cmu = (
          train_ds.shuffle(1024)
          .map(cutmix, num_parallel_calls=AUTO)
          .batch(BATCH_SIZE)
          .prefetch(AUTO)
      )
      #Normal Data Aumentation
      gen = ImageDataGenerator(rotation_range=360,
                              height_shift_range=0.3,
                              width_shift_range=0.3,
                              zoom_range=0.9,
                              horizontal_flip=True,
                              vertical_flip=True, 
                              fill_mode='reflect',
                              brightness_range=[0.5,1.5])
      
      img_shape=Xtrain.shape[1:]

      #Concatenation
      for i, image in tqdm(enumerate(Xtrain)):
        trasformation = gen.get_random_transform(img_shape=img_shape, seed=seed)
        new_image = gen.apply_transform(image, trasformation)
        new_image = np.expand_dims(new_image, axis = 0)
        target = np.expand_dims(Ytrain[i], axis = 0)
        
        Xtrain = np.concatenate((Xtrain, new_image), axis = 0)
        Ytrain = np.concatenate((Ytrain, target), axis = 0)

    
      for image_batch, label_batch in tqdm(iter(train_ds_cmu)):
          Xtrain = np.concatenate((Xtrain, image_batch), axis = 0)
          Ytrain = np.concatenate((Ytrain, label_batch), axis = 0)

      np.save('imagesDA.npy', Xtrain)
      np.save('targetsDA.npy', Ytrain)
        
    print("\ntrain pre-augm  size: "+shapeBeforeDA)
    print("train post-augm size: "+str(Xtrain.shape)+" "+str(Ytrain.shape))
    print("test            size: "+str(Xtest.shape)+" "+str(Ytest.shape))
    print("val             size: "+str(Xval.shape)+" "+str(Yval.shape)) 
    print("val tuner       size: "+str(XvalTuner.shape)+" "+str(YvalTuner.shape)) 

    return Xtest, Ytest, Xtrain, Ytrain, Xval, Yval, XvalTuner, YvalTuner

def build_model(hp):

    supernet = tfk.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,
    classifier_activation="softmax"
    )
    
    inputs = tfk.Input(shape=(input_shape))
    x = supernet( inputs )

    if hp.Boolean("global?"):
      x = tfkl.GlobalAveragePooling2D(name='gap')(x)
    else:
      x = tfkl.Flatten(name='Flattening')(x)
    
    x = tfkl.Dropout(0.3, seed=seed)(x)

    x = tfkl.Dense( 
        units = hp.Int("units", min_value=32, max_value=512, step=32),
        activation='relu',
        kernel_initializer = tfk.initializers.HeUniform(seed))(x)

      # Tune whether to use dropout.
    if hp.Boolean("dropout"):
          x = tfkl.Dropout(0.3, seed=seed )(x)
    
    outputs = tfkl.Dense(
        8, 
        activation='softmax',
        kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


    # Connect input and output through the Model class
    model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    # Compile the model
    model = compile_model(model,hp)
                      
    # Return the model
    return model

def compile_model(model,hp):
    momentum = 0.9
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-3, sampling="log")
    
    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate), metrics='accuracy')

    # Compile the model with a SGD/momentum optimizer
    #model.compile(loss='binary_crossentropy', optimizer=tfk.optimizers.SGD(learning_rate=learning_rate, momentum=momentum), metrics=['accuracy'])

    # Return the model
    return model


# Model Selection

In [ ]:
paramSplit = 0.3
seed = 42
batchSize = BATCH_SIZE
num_folds = 10
path = "dataset"

hp = keras_tuner.HyperParameters()
tfk = tf.keras
tfkl = tf.keras.layers

labels = ['Species1',       # 0
          'Species2',       # 1
          'Species3',       # 2
          'Species4',       # 3
          'Species5',       # 4
          'Species6',       # 5
          'Species7',       # 6
          'Species8' ]      # 7

# Random seed for reproducibility
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Load data from folder
data = load_data(path, labels)

# Spitting data and normalization
Xtest, Ytest, Xtrain, Ytrain, Xval, Yval, XvalTuner, YvalTuner = processing_data(data,labels)


Found 3542 images belonging to 8 classes.
total size: (4054, 96, 96, 3) (4054, 8)

train pre-augm  size: (2837, 96, 96, 3) (2837, 8)
train post-augm size: (8511, 96, 96, 3) (8511, 8)
test            size: (405, 96, 96, 3) (405, 8)
val             size: (406, 96, 96, 3) (406, 8)
val tuner       size: (406, 96, 96, 3) (406, 8)


In [ ]:
input_shape = Xtrain.shape[1:]
epochs = 3

#Fine tuning &/or tranfer learning
base_model_name = 'vgg16'

base_model = tfk.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,
    classifier_activation="softmax"
)

#Setting tuner
tuner = keras_tuner.BayesianOptimization( #Can be RandomSearch, BayesianOptimization or Hyperband
    hypermodel=build_model,
    objective="val_accuracy", 
    max_trials=20,
    executions_per_trial=3,
    overwrite = True, # False = restore result / True = new run
    directory="Tuner", 
    project_name="RIC tuner VGG16",
)

# Building model
model = build_model(hp)

visualkeras.layered_view(base_model, legend=True, spacing=20, scale_xy=5)

# Search for the best hyperparameter
tuner.search(
    x = Xtrain,
    y = Ytrain,
    batch_size = batchSize,
    epochs = epochs,
    validation_data = (XvalTuner, YvalTuner),
)


tuner.results_summary()

Trial 20 Complete [00h 03m 49s]
val_accuracy: 0.5829228162765503

Best val_accuracy So Far: 0.6026272575060526
Total elapsed time: 01h 20m 09s
Results summary
Results in Tuner/RIC tuner VGG16
Showing 10 best trials
Trial summary
Hyperparameters:
global?: False
units: 32
dropout: False
learning_rate: 0.0001
Score: 0.6026272575060526
Trial summary
Hyperparameters:
global?: False
units: 32
dropout: False
learning_rate: 0.0001
Score: 0.6009852091471354
Trial summary
Hyperparameters:
global?: False
units: 32
dropout: False
learning_rate: 0.0001
Score: 0.5977011521657308
Trial summary
Hyperparameters:
global?: False
units: 32
dropout: False
learning_rate: 0.0001
Score: 0.5960591038068136
Trial summary
Hyperparameters:
global?: False
units: 32
dropout: False
learning_rate: 0.0001
Score: 0.5944170753161112
Trial summary
Hyperparameters:
global?: False
units: 32
dropout: False
learning_rate: 0.0001
Score: 0.5903119842211405
Trial summary
Hyperparameters:
global?: False
units: 32
dropout: False


In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]

# Build the model.
best_model.build(input_shape=input_shape)
best_model.summary()

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 Flattening (Flatten)        (None, 4608)              0         
                                                                 
 dropout (Dropout)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 32)                147488    
                                                                 
 dense_1 (Dense)             (None, 8)                 264       
                                                                 
Total params: 14,862,440
Trainable params: 14,862,440
Non-tra

# Training model

In [ ]:
epochs = 400

model = best_model

# Freeze N layers
setNotTrainable = [0, 1, 2, 3, 4, 5, 6]

model.get_layer(base_model_name).trainable = True

for i, layer in enumerate(model.get_layer(base_model_name).layers):
  if i in setNotTrainable:
    layer.trainable = False


# Compile the model
model = compile_model(model,best_hps[0])
model.summary()

# Fitting model
history = model.fit(
    x = Xtrain,
    y = Ytrain,
    batch_size = batchSize,
    epochs = int(epochs/10),
    validation_data = (Xval, Yval),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history


# Sfreeze N layers
model.get_layer(base_model_name).trainable = True


# Compile the model
model = compile_model(model,best_hps[0]) 
model.summary()

# Second Fitting model
history = model.fit(
    x = Xtrain,
    y = Ytrain,
    batch_size = batchSize,
    epochs = epochs,
    validation_data = (Xval, Yval),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 Flattening (Flatten)        (None, 4608)              0         
                                                                 
 dropout (Dropout)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 32)                147488    
                                                                 
 dense_1 (Dense)             (None, 8)                 264       
                                                                 
Total params: 14,862,440
Trainable params: 14,602,280
Non-tra

In [ ]:
# Plot the training
plt.figure(figsize=(20,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(20,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

# Compute the confusion matrix
predictions = model.predict(Xtest)
cm = confusion_matrix(np.argmax(Ytest, axis=-1), np.argmax(predictions, axis=-1))

#Kfold validation
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
score = model.evaluate(Xtest, Ytest)

# Compute the classification metrics
accuracy = accuracy_score(np.argmax(Ytest, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(Ytest, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(Ytest, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(Ytest, axis=-1), np.argmax(predictions, axis=-1), average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))
print('Loss:', score[0])

plt.figure(figsize=(10,8))
# Plot the confusion matrix
sns.heatmap(cm.T, xticklabels=list(labels), yticklabels=list(labels))
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()



In [ ]:
%cd /gdrive/MyDrive/Good\ model
model.save("ModelAccuracy"+("{:.1f}".format(score[1]*100))+"%"+base_model_name)

/gdrive/Othercomputers/Il mio MacBook Air/Challenge1/Good model
